# An Optimization of Class Re-organization Problem

In [1]:
# Import Library
import gurobi as gp
from gurobi import GRB
import pandas as pd
import numpy as np

In [2]:
# Read data from csv file for each class
classes = []
files = ['class1_global.csv', 'class2_global.csv', 'class3_global.csv']
for file in files:
    class_df = pd.read_csv(file)
    classes.append(class_df)

## Notations
&emsp;$ j \in J$ number of old classes<br>
&emsp;$ k \in K$ number of new classes<br>
&emsp;$ N_j$ total number of student in class $j$<br>
&emsp;$ i \in I_j$ student number in class $j$<br>
&emsp;$ w_s, w_p, w_m$ weights of study, sport, musical scores<br>


In [3]:
# Count the number of student in each class
students = [list(classes[i]['Student'].values) for i in range(len(classes))]
N = [len(students[i]) for i in range(len(students))]

#  Setting global variable
NEW_CLASS_NUM = 3
old_class = list(range(1, len(classes)+1))
new_class = list(range(1, NEW_CLASS_NUM+1))
w_stu = 1
w_phy = 1
w_mus = 1


## Notations (2)
&emsp;$ s_{ij}$ study score of student $i$ in class $j$ (3 ranks)<br>
&emsp;$ p_{ij}$ sport score of student $i$ in class $j$ (3 ranks)<br>
&emsp;$ m_{ij}$ musical score of student $i$ in class $j$ (3 ranks)<br>
&emsp;$ r_{ij}$ relay member ability of student $i$ in class $j$ (1: Yes, 0: No)<br>
&emsp;$ h_{ij}$ marathon ability of student $i$ in class $j$ (1: Yes, 0: No)<br>
&emsp;$ n_{ij}$ piano ability of student $i$ in class $j$ (1: Yes, 0: No)<br>
&emsp;$ l_{ij}$ leadership of student $i$ in class $j$ (1: Yes, 0: No)<br>
&emsp;$ g_{ij}$ gender of student $i$ in class $j$ (1: Male, 0: Female)<br>
&emsp;$ t_{ij}$ special attention of student $i$ in class $j$ (1: Yes, 0: No)<br>








In [4]:
# Get the value from dataframe
study = [list(classes[i]['Study'].values) for i in range(len(classes))]
sport = [list(classes[i]['Physical'].values) for i in range(len(classes))]
music = [list(classes[i]['Musical'].values) for i in range(len(classes))]
leadership = [list(classes[i]['Leadership'].values) for i in range(len(classes))]
gender = [list(classes[i]['Gender'].values) for i in range(len(classes))]
piano = [list(classes[i]['Piano'].values) for i in range(len(classes))]
star = [list(classes[i]['Star'].values) for i in range(len(classes))]
relay = [list(classes[i]['Relay'].values) for i in range(len(classes))]
marathon = [list(classes[i]['Marathon'].values) for i in range(len(classes))]
pair = [list(classes[i]['Pair'].values) for i in range(len(classes))]
unpair = [list(classes[i]['Unpair'].values) for i in range(len(classes))]


# Map values score for student
score_study = {(students[j][i], old_class[j], new_class[k]) : study[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_sport = {(students[j][i], old_class[j], new_class[k]) : sport[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_music = {(students[j][i], old_class[j], new_class[k]) : music[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_leadership = {(students[j][i], old_class[j], new_class[k]) : leadership[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_gender = {(students[j][i], old_class[j], new_class[k]) : gender[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_piano = {(students[j][i], old_class[j], new_class[k]) : piano[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_star = {(students[j][i], old_class[j], new_class[k]) : star[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_relay = {(students[j][i], old_class[j], new_class[k]) : relay[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_marathon = {(students[j][i], old_class[j], new_class[k]) : marathon[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_pair = {(students[j][i], old_class[j], new_class[k]) : pair[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}
score_unpair = {(students[j][i], old_class[j], new_class[k]) : unpair[j][i] for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))}

In [5]:
# Declare and initialize model
m = gp.Model('RAP')

Using license file /Users/thitiwat/gurobi.lic
Academic license - for non-commercial use only


## Define decision variables
&emsp;$ x_{ijk} \begin{cases} 1, &\text{if students } i \text{ in old class } j \text{ is assigned to new class } k\\ 0, & \text{otherwise}\end{cases}$<br>
&emsp;$ \mu_s, \mu_p, \mu_m$ maximum valuse from sum of subject scores in each class<br>
&emsp;$ \lambda_s, \lambda_p, \lambda_m$ maximum valuse from sum of subject scores in each class<br>

In [6]:
# Add variable to model
x = m.addVars([(students[j][i], old_class[j], new_class[k]) 
    for j in range(len(old_class)) for i in range(N[j]) for k in range(len(new_class))], name='x', vtype='B')   # x is binary
x

{(1, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (3, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (3, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (4, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (4, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (4, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (5, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (5, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (5, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (6, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (6, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (6, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (7, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (7, 1, 2): <gurobi.Var *Awaiting Model Update*>,


In [7]:
# max-min study score
ms = m.addVar()
ss = m.addVar()
mp = m.addVar()
sp = m.addVar()
mm = m.addVar()
sm = m.addVar()

## Constraints
A student only assigned for 1 class
$$ \sum _{k} x_{ijk} = 1 ,\forall i \in I, \forall i \in J $$


In [8]:
class_old = m.addConstrs((x.sum(i, j, '*') == 1 for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)), name='old_class')
class_old

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model

Balance of reorganization in each class
$$ \Big\lfloor \frac{N_j}{K} \Big\rfloor \le \sum _{i} x_{ijk} \le \Big\lfloor \frac{N_j}{K} \Big\rfloor + 1,\forall j \in J, \forall k \in K $$

In [9]:
class_reor_max = m.addConstrs((x.sum('*', j, k) <= int(N[j-1]/len(new_class)) + 1 \
                            for j in range(1, len(old_class)+1) for k in new_class), name='reor_class_max')
class_reor_min = m.addConstrs((x.sum('*', j, k) >= int(N[j-1]/len(new_class)) \
                            for j in range(1, len(old_class)+1) for k in new_class), name='reor_class_min')

Balance total number of student between classes
$$ \Big\lfloor \frac{\sum_{j} N_j}{K} \Big\rfloor \le \sum_{j}\sum _{i} x_{ijk} \le \Big\lfloor \frac{\sum_{j} N_j}{K} \Big\rfloor + 1, \forall k \in K $$

In [10]:
class_new_max = m.addConstrs((x.sum('*','*', k) <= int(sum(N)/len(new_class)) + 1 for k in new_class), name='new_class_max')
class_new_min = m.addConstrs((x.sum('*','*', k) >= int(sum(N)/len(new_class)) for k in new_class), name='new_class_min')

Sum of score is limit in the range of upper bound and lower bound
$$ \lambda_s \le \sum _{i} s_i x_{ij} \le \lambda_s, \forall k \in K $$
$$ \lambda_p \le \sum _{i} p_i x_{ij} \le \lambda_p, \forall k \in K $$
$$ \lambda_m \le \sum _{i} m_i x_{ij} \le \lambda_m, \forall k \in K $$


In [11]:
max_study = m.addConstrs((gp.quicksum(x[i,j,k] * score_study[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) <= ms for k in new_class), name='maxs')
min_study = m.addConstrs((gp.quicksum(x[i,j,k] * score_study[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= ss for k in new_class), name='mins')
max_sport = m.addConstrs((gp.quicksum(x[i,j,k] * score_sport[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) <= mp for k in new_class), name='maxsp')
min_sport = m.addConstrs((gp.quicksum(x[i,j,k] * score_sport[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= sp for k in new_class), name='minsp')
max_music = m.addConstrs((gp.quicksum(x[i,j,k] * score_music[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) <= mm for k in new_class), name='maxm')
min_music = m.addConstrs((gp.quicksum(x[i,j,k] * score_music[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= sm for k in new_class), name='minm')

At least one leadership student is assigned for each class
$$ \sum_j \sum_i l_{ij} x_{ijk} \ge 1, \forall k \in K$$
At least one piano ability student is assigned for each class
$$ \sum_j \sum_i n_{ij} x{ijk} \ge 1, \forall k \in K$$

In [12]:
lead = m.addConstrs((gp.quicksum(x[i,j,k] * score_leadership[i,j,k] 
                        for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= 1 for k in new_class), name='laed')
piano = m.addConstrs((gp.quicksum(x[i,j,k] * score_piano[i,j,k] 
                        for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= 1 for k in new_class), name='piano')

Balance sex of student between classes
$$ \Big\lfloor \frac{\sum_j \sum_i g_{ij}}{K} \Big\rfloor \le \sum_{j} \sum_{i} g_{ij} x_{ijk} \le \Big\lfloor \frac{\sum_j \sum_i g_{ij}}{K} \Big\rfloor + 1, \forall k \in K $$

In [13]:
all_gen = 0
for j in range(len(old_class)):
    all_gen += sum(gender[j])
gen_max = m.addConstrs((gp.quicksum(x[i,j,k] * score_gender[i,j,k] 
                        for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) <= int(all_gen/len(new_class)) + 1 for k in new_class), 
                        name='gender_max')
gen_min = m.addConstrs((gp.quicksum(x[i,j,k] * score_gender[i,j,k] 
                        for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= int(all_gen/len(new_class)) for k in new_class), 
                        name='gender_min')

Balance special attention of student between classes
$$ \Big\lfloor \frac{\sum_j \sum_i t_{ij}}{K} \Big\rfloor \le \sum_{j} \sum_{i} t_{ij} x_{ijk} \le \Big\lfloor \frac{\sum_j \sum_i t_{ij}}{K} \Big\rfloor + 1, \forall k \in K $$

In [14]:
all_star = 0
for j in range(len(old_class)):
    all_star += sum(star[j])
star_max = m.addConstrs((gp.quicksum(x[i,j,k] * score_star[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) <= int(all_star/len(new_class)) + 1 for k in new_class), 
                            name='star_max')
star_min = m.addConstrs((gp.quicksum(x[i,j,k] * score_star[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= int(all_star/len(new_class)) for k in new_class), 
                            name='star_min')

Relay member for boy and girls in each class
$$ \sum_j \sum_i g_{ij} r_{ij} x_{ijk} \ge 4, \forall k \in K$$
$$ \sum_j \sum_i (1 - g_{ij}) r_{ij} x_{ijk} \ge 4, \forall k \in K$$

In [15]:
relay_boy = m.addConstrs((gp.quicksum(x[i,j,k] * score_relay[i,j,k] * score_gender[i,j,k] 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= 4 for k in new_class), name='relay_b')
relay_girl = m.addConstrs((gp.quicksum(x[i,j,k] * score_relay[i,j,k] * (1-score_gender[i,j,k]) 
                            for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= 4 for k in new_class), name='relay_g')

Balance marathon student for each class
$$ \Big\lfloor \frac{\sum_j \sum_i h_{ij}}{K} \Big\rfloor \le \sum_{j} \sum_{i} h_{ij} x_{ijk} \le \Big\lfloor \frac{\sum_j \sum_i h_{ij}}{K} \Big\rfloor + 1, \forall k \in K $$

In [16]:
all_marathon = 0
for j in range(len(old_class)):
    all_marathon += sum(marathon[j])
marathon_max = m.addConstrs((gp.quicksum(x[i,j,k] * score_marathon[i,j,k] 
                                for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) <= int(all_marathon/len(new_class)) + 1 for k in new_class), 
                                name='mara_max')
marathon_min = m.addConstrs((gp.quicksum(x[i,j,k] * score_marathon[i,j,k] 
                                for j in range(1, len(old_class)+1) for i in range(1, N[j-1]+1)) >= int(all_marathon/len(new_class)) for k in new_class), 
                                name='mara_min')

Add constraint for pair student
$$ x_{ijk} = x_{ljk} , \forall j \in J, \forall k \in K, \forall i \in P_i, \forall l \in P_l$$

In [17]:
# append data from pair stdents
a = []
for j in old_class:
    l = [(x+1, y+1) for x in range(N[j-1]) for y in range(N[j-1]) if pair[j-1][x] == pair[j-1][y] and pair[j-1][x] > 0 and pair[j-1][y]>0 and x!=y]
    data = {tuple(sorted(item)) for item in l}
    data = list(data)
    a.append(data)
i_1_j = []
i_2_j = []
i_1 = []
i_2 = []
for i in a:
    for s in i:
        i_1.append(s[0])
        i_2.append(s[1])
    i_1_j.append(i_1)
    i_2_j.append(i_2)
    i_1=[]
    i_2=[]

In [18]:
pair_con = m.addConstrs(((x[i,j,k]) == (x[l,j,k]) 
                            for j in range(1, len(old_class)+1) for i in i_1_j[j-1] for l in i_2_j[j-1] for k in new_class), name='pair')        

Add constraint for unpair student
$$ x_{ijk} * x_{ljk} = 0, \forall j \in J, \forall k \in K, \forall i \in U_i, \forall l \in U_l$$

In [19]:
# append data for unpari students
b = []
for j in old_class:
    l = [(x+1, y+1) for x in range(N[j-1]) for y in range(N[j-1]) if unpair[j-1][x] == unpair[j-1][y] and unpair[j-1][x] > 0 and unpair[j-1][y]>0 and x!=y]
    data = {tuple(sorted(item)) for item in l}
    data = list(data)
    b.append(data)
ui_1_j = []
ui_2_j = []
ui_1 = []
ui_2 = []
for i in b:
    for s in i:
        ui_1.append(s[0])
        ui_2.append(s[1])
    ui_1_j.append(ui_1)
    ui_2_j.append(ui_2)
    ui_1=[]
    ui_2=[]


In [20]:
unpair_con = m.addConstrs(((x[i,j,k]) * (x[l,j,k]) == 0 
                            for j in range(1, len(old_class)+1) for i in ui_1_j[j-1] for l in ui_2_j[j-1] for k in new_class), name='unpair')


Set the objectiva function to minimize the weighted differnce between maximal and minimal score of each ability score between classes
$$ min (\mu_s - \lambda_s)w_s + (\mu_p - \lambda_p)w_p + (\mu_m - \lambda_m)w_m

In [21]:
m.setObjective((ms-ss)*w_stu + (mp-sp)*w_phy + (mm-sm)*w_mus,  GRB.MINIMIZE)
m.write('RAP_Global.lp')
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 171 rows, 276 columns and 3498 nonzeros
Model fingerprint: 0x477f8787
Model has 9 quadratic constraints
Variable types: 6 continuous, 270 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 6 rows and 9 columns
Presolve time: 0.01s
Presolved: 165 rows, 267 columns, 3360 nonzeros
Variable types: 0 continuous, 267 integer (261 binary)

Root relaxation: objective -1.421085e-14, 230 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.00000    0   25          -   -0.00000      -     -    0s
H    0     0                       6.0000000   -0.00000   100%     -    0s
H    0     0                      

In [22]:
# # Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# # Display optimal total matching score
print('Total matching score: ', m.objVal)

x[1,1,3] 1.0
x[2,1,1] 1.0
x[3,1,1] 1.0
x[4,1,2] 1.0
x[5,1,2] 1.0
x[6,1,3] 1.0
x[7,1,3] 1.0
x[8,1,3] 1.0
x[9,1,3] 1.0
x[10,1,2] 1.0
x[11,1,2] 1.0
x[12,1,3] 1.0
x[13,1,1] 1.0
x[14,1,3] 1.0
x[15,1,1] 1.0
x[16,1,2] 1.0
x[17,1,1] 1.0
x[18,1,1] 1.0
x[19,1,2] 1.0
x[20,1,3] 1.0
x[21,1,2] 1.0
x[22,1,1] 1.0
x[23,1,3] 1.0
x[24,1,3] 1.0
x[25,1,2] 1.0
x[26,1,2] 1.0
x[27,1,1] 1.0
x[28,1,2] 1.0
x[29,1,1] 1.0
x[30,1,1] 1.0
x[1,2,1] 1.0
x[2,2,1] 1.0
x[3,2,1] 1.0
x[4,2,1] 1.0
x[5,2,3] 1.0
x[6,2,1] 1.0
x[7,2,2] 1.0
x[8,2,2] 1.0
x[9,2,2] 1.0
x[10,2,1] 1.0
x[11,2,1] 1.0
x[12,2,3] 1.0
x[13,2,3] 1.0
x[14,2,1] 1.0
x[15,2,2] 1.0
x[16,2,3] 1.0
x[17,2,3] 1.0
x[18,2,3] 1.0
x[19,2,2] 1.0
x[20,2,3] 1.0
x[21,2,1] 1.0
x[22,2,2] 1.0
x[23,2,2] 1.0
x[24,2,3] 1.0
x[25,2,3] 1.0
x[26,2,2] 1.0
x[27,2,1] 1.0
x[28,2,2] 1.0
x[29,2,2] 1.0
x[30,2,3] 1.0
x[1,3,2] 1.0
x[2,3,3] 1.0
x[3,3,2] 1.0
x[4,3,1] 1.0
x[5,3,2] 1.0
x[6,3,3] 1.0
x[7,3,3] 1.0
x[8,3,2] 1.0
x[9,3,3] 1.0
x[10,3,3] 1.0
x[11,3,1] 1.0
x[12,3,2] 1.0
x[13,3,1] 1.0
x[14,

In [23]:
# Create table for assign student to new class
table = []
for j in range(1, len(old_class)+1):
    for i in range(1, N[j-1]+1):
        for k in new_class:
            if(x[i, j, k].X == 1):
                table.append([i,j,k])
table

[[1, 1, 3],
 [2, 1, 1],
 [3, 1, 1],
 [4, 1, 2],
 [5, 1, 2],
 [6, 1, 3],
 [7, 1, 3],
 [8, 1, 3],
 [9, 1, 3],
 [10, 1, 2],
 [11, 1, 2],
 [12, 1, 3],
 [13, 1, 1],
 [14, 1, 3],
 [15, 1, 1],
 [16, 1, 2],
 [17, 1, 1],
 [18, 1, 1],
 [19, 1, 2],
 [20, 1, 3],
 [21, 1, 2],
 [22, 1, 1],
 [23, 1, 3],
 [24, 1, 3],
 [25, 1, 2],
 [26, 1, 2],
 [27, 1, 1],
 [28, 1, 2],
 [29, 1, 1],
 [30, 1, 1],
 [1, 2, 1],
 [2, 2, 1],
 [3, 2, 1],
 [4, 2, 1],
 [5, 2, 3],
 [6, 2, 1],
 [7, 2, 2],
 [8, 2, 2],
 [9, 2, 2],
 [10, 2, 1],
 [11, 2, 1],
 [12, 2, 3],
 [13, 2, 3],
 [14, 2, 1],
 [15, 2, 2],
 [16, 2, 3],
 [17, 2, 3],
 [18, 2, 3],
 [19, 2, 2],
 [20, 2, 3],
 [21, 2, 1],
 [22, 2, 2],
 [23, 2, 2],
 [24, 2, 3],
 [25, 2, 3],
 [26, 2, 2],
 [27, 2, 1],
 [28, 2, 2],
 [29, 2, 2],
 [30, 2, 3],
 [1, 3, 2],
 [2, 3, 3],
 [3, 3, 2],
 [4, 3, 1],
 [5, 3, 2],
 [6, 3, 3],
 [7, 3, 3],
 [8, 3, 2],
 [9, 3, 3],
 [10, 3, 3],
 [11, 3, 1],
 [12, 3, 2],
 [13, 3, 1],
 [14, 3, 2],
 [15, 3, 2],
 [16, 3, 1],
 [17, 3, 3],
 [18, 3, 1],
 [19, 3, 2],


In [24]:
df = pd.DataFrame(table, columns=['Student No.', 'Old class', 'New class'])
new_excel = df.to_excel('assignment.xlsx', sheet_name='to_excel')